## 3. Clean Dataset

### 3.1. Extract Data

In [ ]:
# Ignore data that is not relevant to commodites.   
comms = comms[11:-2]
# Hyperlinks of corresponding commodities.
hrefs = hrefs[11:-2]

date = []
price = []

for href in hrefs: 
    
    # Extract data from the subpage of corresponding commoditiy. . 
    resp = requests.get(href)
    resp.encoding='utf-8'
    soup = BeautifulSoup(resp.text, 'html.parser')
    
    # Price for each trading session of the specified commodity is listed in:
    # Level 1: 3rd Tag "table" with Class named "indexpagetable."
    # Level 2: Tag "tr" with Class named "row1" and "row2,"
    # where "row1" stands for "odd rows" of the table
    # while "row2" stands for "even rows" of the table.
    # Level 3: 1st and 4th Tag "td" include the "dates" of trading sessions
    # while 2nd and 5th Tag "td" include the corresponding prices.
    tables = soup.find_all('table', {'class': 'indexpagetable'})
    table = tables[2]

    rows = table.find_all('tr', {'class': ['row1', 'row2']})    

    # .find: find the 1st one.
    for row in rows:
        td = row.find('td')
        date.append(td.text)   

    for row in rows:
        td = row.find_all('td')[3]
        date.append(td.text)

    for row in rows:
        td = row.find_all('td')[1]
        price.append(td.text)

    for row in rows:
        td = row.find_all('td')[4]
        price.append(td.text)  
    
    # Make the length of "List of Commodities" identical to that "Date" and "Price" data
    # by repeating each commodity itself 20 times.   
    n = int(len(date)/len(comms))
    comm = list(itertools.chain.from_iterable((itertools.repeat(comm, n) for comm in comms)))

### 3.2. Create Dataframe

In [ ]:
df = pd.DataFrame({'comm':comm, 'date': date, 'price': price})
df

### 3.3. Remove Data

In [ ]:
# Specify commodies that are not within the analysis period.  
df[df['date'].str.contains('2018/09')]

In [ ]:
# Exclude the specified data from the dataframe.
df = df[df['comm'] != '鈾']
df

In [ ]:
# Remove the specified commodity from the list.
comms.remove('鈾')
comms

### 3.4. Split Dataframe into Chunks

In [ ]:
# There are 20 datasets for each commodity.  
n = int(len(date)/len(comms))
# There are 540 rows in "df" and the interval should be set as 20 to split "df" into 27 even chunks. 
list_df = [df[i:i+n] for i in range(0,df.shape[0],n)]
list_df